In [6]:
import pandas as pd

df_superstore =  pd.read_csv('superstore_dataset.csv', encoding='latin1')

# Renombrando los nombres de las columnas para poder acceder de mejor manera 
df_superstore.columns = [
    col.lower().replace(' ', '_').replace('-','_')
    for col in df_superstore.columns
]

# función para ver los typos de cada columna de un DataFrame
def revisatipos (df):
        for col in df:
            print (f'El tipo de {col} es: ',df[col].dtype)
        
df_superstore.head()


,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,...,postal_code,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [7]:
# eleimino las columnas que no me aportan información y las que no usaré 
df_superstore_limpio = df_superstore.drop(columns=['country','row_id'])

# chequeando los tipos de dato de cada columna y modificando de ser necesario

#revisatipos(df_superstore_limpio)

for col in ['ship_mode', 'segment','city','state','region', 'region','category','sub_category']:
    df_superstore_limpio[col] = df_superstore_limpio[col].astype('category') # modifico los tipo object a category para optimizar 

df_superstore_limpio['order_date'] = pd.to_datetime(df_superstore_limpio['order_date'])
df_superstore_limpio['ship_date'] = pd.to_datetime(df_superstore_limpio['ship_date'])
df_superstore_limpio['postal_code'] = df_superstore_limpio['postal_code'].astype('Int64')

revisatipos(df_superstore_limpio)


El tipo de order_id es:  object
El tipo de order_date es:  datetime64[ns]
El tipo de ship_date es:  datetime64[ns]
El tipo de ship_mode es:  category
El tipo de customer_id es:  object
El tipo de customer_name es:  object
El tipo de segment es:  category
El tipo de city es:  category
El tipo de state es:  category
El tipo de postal_code es:  Int64
El tipo de region es:  category
El tipo de product_id es:  object
El tipo de category es:  category
El tipo de sub_category es:  category
El tipo de product_name es:  object
El tipo de sales es:  float64
El tipo de quantity es:  int64
El tipo de discount es:  float64
El tipo de profit es:  float64


In [8]:
df_superstore_limpio.head()

,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,city,state,postal_code,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit
0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [15]:
# empieza el proceso para desnormalizar con el objetivo pasarla a Power BI 
# Tabla de products 

df_products = df_superstore_limpio[[
    'product_id',
    'product_name', 
    'category', 
    'sub_category'
]].drop_duplicates()

revisatipos(df_products)

# agregando un groupby acá para no tener problemas en PowerBi con las relaciones entre tablas, para que su relación sea de 1 a muchos (1:*)
df_products = df_products.groupby('product_id', as_index=False).first()

df_products.columns



El tipo de product_id es:  object
El tipo de product_name es:  object
El tipo de category es:  category
El tipo de sub_category es:  category


Index(['product_id', 'product_name', 'category', 'sub_category'], dtype='object')

In [14]:
# Tabla de customers
df_customers = df_superstore_limpio[[
    'customer_id',
    'customer_name',
    'segment'
]].drop_duplicates()

revisatipos(df_customers)

df_customers.columns

El tipo de customer_id es:  object
El tipo de customer_name es:  object
El tipo de segment es:  category


Index(['customer_id', 'customer_name', 'segment'], dtype='object')

In [13]:
# tabla de orders
df_orders = df_superstore_limpio[[
    'order_id',
    'order_date',
    'ship_date',
    'ship_mode',
    'sales',
    'quantity',
    'discount',
    'profit',
    'customer_id',
    'product_id',
    'city',
    'state',
    'region',
    'postal_code'
]].drop_duplicates()

revisatipos(df_orders)

df_orders.columns

El tipo de order_id es:  object
El tipo de order_date es:  datetime64[ns]
El tipo de ship_date es:  datetime64[ns]
El tipo de ship_mode es:  category
El tipo de sales es:  float64
El tipo de quantity es:  int64
El tipo de discount es:  float64
El tipo de profit es:  float64
El tipo de customer_id es:  object
El tipo de product_id es:  object
El tipo de city es:  category
El tipo de state es:  category
El tipo de region es:  category
El tipo de postal_code es:  Int64


Index(['order_id', 'order_date', 'ship_date', 'ship_mode', 'sales', 'quantity',
       'discount', 'profit', 'customer_id', 'product_id', 'city', 'state',
       'region', 'postal_code'],
      dtype='object')

In [12]:
# chqeueo que no hayan duplicados ni nada raro
print(
    f'''
    La cantidad de filas duplicadas de products son: {df_products.duplicated().sum()}, de nulos: \n{df_products.isnull().sum()},
    La cantidad de filas duplicadas de customers son: {df_customers.duplicated().sum()} de nulos: \n{df_customers.isnull().sum()},
    La cantidad de filas duplicadas de orders son: {df_orders.duplicated().sum()} de nulos: \n{df_orders.isnull().sum()}
    '''
) # voy a ignorar los nulos en postal_code en este caso 

# cargo los csv para pasarlos a Power BI 

#df_products.to_csv('products.csv', index=False)
#df_customers.to_csv('customers.csv', index=False)
#df_orders.to_csv('orders.csv', sep=';', decimal=',' , index=False)


    La cantidad de filas duplicadas de products son: 0, de nulos: 
product_id      0
product_name    0
category        0
sub_category    0
dtype: int64,
    La cantidad de filas duplicadas de customers son: 0 de nulos: 
customer_id      0
customer_name    0
segment          0
dtype: int64,
    La cantidad de filas duplicadas de orders son: 0 de nulos: 
order_id       0
order_date     0
ship_date      0
ship_mode      0
sales          0
quantity       0
discount       0
profit         0
customer_id    0
product_id     0
city           0
state          0
region         0
postal_code    0
dtype: int64
    
